<a href="https://colab.research.google.com/github/YugyeongJo/study_data_analytics/blob/main/docs/quests/CDAs/CDA_mixed_unNormal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
### 한글 폰트 설치
!apt-get install -y fonts-nanum
!fc-cache -fv
!rm ~/.cache/matplotlib -rf
# 설치 후 colab의 경우 Runtime > Restart session 필요

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
fonts-nanum is already the newest version (20200506-1).
0 upgraded, 0 newly installed, 0 to remove and 32 not upgraded.
/usr/share/fonts: caching, new cache contents: 0 fonts, 1 dirs
/usr/share/fonts/truetype: caching, new cache contents: 0 fonts, 3 dirs
/usr/share/fonts/truetype/humor-sans: caching, new cache contents: 1 fonts, 0 dirs
/usr/share/fonts/truetype/liberation: caching, new cache contents: 16 fonts, 0 dirs
/usr/share/fonts/truetype/nanum: caching, new cache contents: 12 fonts, 0 dirs
/usr/local/share/fonts: caching, new cache contents: 0 fonts, 0 dirs
/root/.local/share/fonts: skipping, no such directory
/root/.fonts: skipping, no such directory
/usr/share/fonts/truetype: skipping, looped directory detected
/usr/share/fonts/truetype/humor-sans: skipping, looped directory detected
/usr/share/fonts/truetype/liberation: skipping, looped directory detected
/usr/share/fonts/truetype/

In [8]:
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rc('font', family='NanumBarunGothic') # 혹은 다른 설치한 Nanum 폰트 사용

## load dataset

In [44]:
import pandas as pd
import seaborn as sns
import scipy.stats as stats
df_saleshistory = pd.read_csv('/content/fruitstore_saleshistory.csv')
df_saleshistory[:5]

,공급일자,요일,공급월,공급주차,회원번호,조합원상태,물품대분류,물품중분류,물품소분류,물품명,구매수량,주소-구,주소-동,성별,연령,연령대,구매금액,구매매장,반품_원거래일자,구매시각
0,2018-01-02,화,1,1,272369856,정상회원,과실,과일,사과,사과/유(1.5kg),1.0,수지구,풍덕천동,여,45.0,40대,22207,매장C,NaN,10:04
1,2018-01-02,화,1,1,1506656256,정상회원,채소,버섯,느타리버섯,느타리버섯(300g),1.0,수지구,풍덕천동,여,36.0,30대이하,4977,매장C,NaN,10:05
2,2018-01-02,화,1,1,1506656256,정상회원,축산물,알,유정란,유정란/매장용(10알/국내산),1.0,수지구,풍덕천동,여,36.0,30대이하,7083,매장C,NaN,10:05
3,2018-01-02,화,1,1,1023108864,정상회원,반찬,두부/유부,두부,연두부(100g),1.0,수지구,풍덕천동,여,36.0,30대이하,766,매장C,NaN,10:08
4,2018-01-02,화,1,1,1476143616,정상회원,간식,빵,"식빵, 식사대용",아침빵(240g/8개)/매장,1.0,수지구,풍덕천동,여,34.0,30대이하,4403,매장C,NaN,10:09


## 범주형/연속형

### 등분산 : 회원의 연령대(범주형)에 따라 구매 빈도(연속형)의 분산이 동일한지 검증

> 들여쓴 블록



#### 결측치, 이상치 확인

In [46]:
df_saleshistory.isnull().sum()
# 연령대 결측치 33개 무시 가능한 수준으로 판단

df_saleshistory.dropna(subset=['연령대'], inplace=True)
df_saleshistory.isnull().sum()

공급일자             0
요일               0
공급월              0
공급주차             0
회원번호             0
조합원상태            0
물품대분류            0
물품중분류            0
물품소분류            0
물품명              0
구매수량             0
주소-구             0
주소-동             0
성별               5
연령               0
연령대              0
구매금액             0
구매매장             0
반품_원거래일자    694304
구매시각             0
dtype: int64

In [51]:
df_saleshistory['연령대'].unique()

array(['40대', '30대이하', '50대', '60대', '70대이상'], dtype=object)

In [52]:
df_saleshistory['공급일자'].unique()

array(['2018-01-02', '2018-01-03', '2018-01-04', '2018-01-05',
       '2018-01-06', '2018-01-07', '2018-01-08', '2018-01-09',
       '2018-01-10', '2018-01-11', '2018-01-12', '2018-01-13',
       '2018-01-14', '2018-01-15', '2018-01-16', '2018-01-17',
       '2018-01-18', '2018-01-19', '2018-01-20', '2018-01-21',
       '2018-01-22', '2018-01-23', '2018-01-24', '2018-01-25',
       '2018-01-26', '2018-01-27', '2018-01-28', '2018-01-29',
       '2018-01-30', '2018-01-31', '2018-02-01', '2018-02-02',
       '2018-02-03', '2018-02-04', '2018-02-05', '2018-02-06',
       '2018-02-07', '2018-02-08', '2018-02-09', '2018-02-10',
       '2018-02-11', '2018-02-12', '2018-02-13', '2018-02-14',
       '2018-02-15', '2018-02-19', '2018-02-20', '2018-02-21',
       '2018-02-22', '2018-02-23', '2018-02-24', '2018-02-25',
       '2018-02-26', '2018-02-27', '2018-02-28', '2018-03-01',
       '2018-03-02', '2018-03-03', '2018-03-04', '2018-03-05',
       '2018-03-06', '2018-03-07', '2018-03-08', '2018-

In [53]:
df_saleshistory['회원번호'].unique()

array([ 272369856, 1506656256, 1023108864, ..., 1323476480,  433301760,
         26440576])

In [54]:
df_saleshistory['구매시각'].unique()

array(['10:04', '10:05', '10:08', '10:09', '10:10', '10:12', '10:19',
       '10:23', '10:25', '10:29', '10:31', '10:32', '10:37', '10:39',
       '10:40', '10:49', '10:57', '11:07', '11:08', '11:14', '11:18',
       '11:19', '11:20', '11:21', '11:22', '11:23', '11:33', '11:34',
       '11:35', '11:38', '11:39', '11:40', '11:41', '11:44', '11:50',
       '11:51', '11:58', '11:59', '12:09', '12:12', '12:18', '12:19',
       '12:20', '12:23', '12:25', '12:26', '12:29', '12:36', '12:39',
       '12:45', '12:49', '12:55', '13:01', '13:04', '13:12', '13:14',
       '13:15', '13:19', '13:21', '13:24', '13:34', '13:35', '13:36',
       '13:37', '13:40', '13:41', '13:42', '13:46', '13:50', '13:53',
       '13:58', '14:00', '14:02', '14:04', '14:05', '14:06', '14:07',
       '14:18', '14:19', '14:23', '14:24', '14:25', '14:29', '14:32',
       '14:45', '14:47', '14:50', '14:51', '14:55', '14:58', '15:08',
       '15:12', '15:13', '15:19', '15:23', '15:25', '15:26', '15:27',
       '15:29', '15:

#### 구매빈도 확인

In [60]:
# 구매 빈도 = 동일한 공급일자 + 동일한 구매시각 + 동일한 회원번호 인 경우 count
df_saleshistory_purchase_frequency = df_saleshistory.drop_duplicates(subset=['공급일자', '회원번호', '구매시각'], keep='first')
df_saleshistory_purchase_frequency

,공급일자,요일,공급월,공급주차,회원번호,조합원상태,물품대분류,물품중분류,물품소분류,물품명,구매수량,주소-구,주소-동,성별,연령,연령대,구매금액,구매매장,반품_원거래일자,구매시각
0,2018-01-02,화,1,1,272369856,정상회원,과실,과일,사과,사과/유(1.5kg),1.0,수지구,풍덕천동,여,45.0,40대,22207,매장C,NaN,10:04
1,2018-01-02,화,1,1,1506656256,정상회원,채소,버섯,느타리버섯,느타리버섯(300g),1.0,수지구,풍덕천동,여,36.0,30대이하,4977,매장C,NaN,10:05
3,2018-01-02,화,1,1,1023108864,정상회원,반찬,두부/유부,두부,연두부(100g),1.0,수지구,풍덕천동,여,36.0,30대이하,766,매장C,NaN,10:08
4,2018-01-02,화,1,1,1476143616,정상회원,간식,빵,"식빵, 식사대용",아침빵(240g/8개)/매장,1.0,수지구,풍덕천동,여,34.0,30대이하,4403,매장C,NaN,10:09
5,2018-01-02,화,1,1,716128320,정상회원,축산물,알,유정란,유정란/매장용(10알/국내산),1.0,수지구,풍덕천동,여,51.0,50대,7083,매장C,NaN,10:10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
705354,2018-06-30,토,6,26,1553271552,정상회원,과실,과일,황매실,황매실(남고)/유(5kg),2.0,수지구,신봉동,여,45.0,40대,111036,매장D,NaN,12:49
705415,2018-06-30,토,6,26,1515509376,정상회원,축산물,알,유정란,유정란/매장용(15알/국내산),1.0,수지구,상현동,여,73.0,70대이상,10338,매장B,NaN,11:43
705429,2018-06-30,토,6,26,606984384,정상회원,과실,과일,산딸기,냉동산딸기(250g),3.0,수지구,풍덕천동,여,51.0,50대,35608,매장B,NaN,19:34
705469,2018-06-30,토,6,26,639109888,정상회원,주곡,멥쌀,멥쌀,백미/유(8kg)-매장용,1.0,수지구,풍덕천동,여,40.0,40대,58773,매장B,NaN,15:49


#### 1) 정규화 검증

In [63]:
df_saleshistory["연령대"].value_counts()
# 5000개 이상 Anderson test

40대      270573
50대      174458
30대이하    106392
60대       95019
70대이상     59096
Name: 연령대, dtype: int64

In [77]:
df_saleshistory['구매빈도'] = df_saleshistory['공급일자'] +df_saleshistory['회원번호'].astype(str) + df_saleshistory['구매시각']
df_saleshistory

,공급일자,요일,공급월,공급주차,회원번호,조합원상태,물품대분류,물품중분류,물품소분류,물품명,...,주소-구,주소-동,성별,연령,연령대,구매금액,구매매장,반품_원거래일자,구매시각,구매빈도
0,2018-01-02,화,1,1,272369856,정상회원,과실,과일,사과,사과/유(1.5kg),...,수지구,풍덕천동,여,45.0,40대,22207,매장C,NaN,10:04,2018-01-0227236985610:04
1,2018-01-02,화,1,1,1506656256,정상회원,채소,버섯,느타리버섯,느타리버섯(300g),...,수지구,풍덕천동,여,36.0,30대이하,4977,매장C,NaN,10:05,2018-01-02150665625610:05
2,2018-01-02,화,1,1,1506656256,정상회원,축산물,알,유정란,유정란/매장용(10알/국내산),...,수지구,풍덕천동,여,36.0,30대이하,7083,매장C,NaN,10:05,2018-01-02150665625610:05
3,2018-01-02,화,1,1,1023108864,정상회원,반찬,두부/유부,두부,연두부(100g),...,수지구,풍덕천동,여,36.0,30대이하,766,매장C,NaN,10:08,2018-01-02102310886410:08
4,2018-01-02,화,1,1,1476143616,정상회원,간식,빵,"식빵, 식사대용",아침빵(240g/8개)/매장,...,수지구,풍덕천동,여,34.0,30대이하,4403,매장C,NaN,10:09,2018-01-02147614361610:09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
705566,2018-06-30,토,6,26,146297344,정상회원,수산,건어물,멸치류,국물멸치(300g),...,수지구,성복동,여,51.0,50대,9189,매장A,NaN,17:56,2018-06-3014629734417:56
705567,2018-06-30,토,6,26,1532985472,정상회원,수산,해조,미역/다시마,자른다시마(100g),...,수지구,동천동,여,63.0,60대,5169,매장A,NaN,14:37,2018-06-30153298547214:37
705568,2018-06-30,토,6,26,1532985472,정상회원,반찬,두부/유부,두부,두부(1모:420g),...,수지구,동천동,여,63.0,60대,7466,매장A,NaN,14:37,2018-06-30153298547214:37
705569,2018-06-30,토,6,26,1209779776,정상회원,간식,과자,스낵,하늘바다새우(60g),...,수지구,동천동,여,41.0,40대,3542,매장A,NaN,18:04,2018-06-30120977977618:04


In [91]:
saleshitory_pivot =pd.pivot_table(data=df_saleshistory, index=['연령대','회원번호'], values='구매빈도', aggfunc='count')
saleshitory_pivot=saleshitory_pivot.reset_index().set_index('회원번호')
saleshitory_pivot

,연령대,구매빈도
회원번호,,
76160960,30대이하,2
120020096,30대이하,7
139359616,30대이하,24
164094528,30대이하,44
185479360,30대이하,24
...,...,...
1597618816,70대이상,19
1601369792,70대이상,27
1605625600,70대이상,2


In [92]:
# stats.anderson을 사용한 정규 분포 검정
result = stats.anderson(saleshitory_pivot['구매빈도'])
print('Test Statistic: %.3f' % result.statistic)
for i in range(len(result.critical_values)):
    sl, cv = result.significance_level[i], result.critical_values[i]
    if result.statistic < cv:
        print('%.1f%% 수준에서 데이터는 정규 분포를 따름 (검정 통계량 %.3f < 임계값 %.3f)' % (sl, result.statistic, cv))
    else:
        print('%.1f%% 수준에서 데이터는 정규 분포를 따르지 않음 (검정 통계량 %.3f >= 임계값 %.3f)' % (sl, result.statistic, cv))
# 비정규분포 / 범주형 2집단 Wilcoxon Test 사용

Test Statistic: 1383.026
15.0% 수준에서 데이터는 정규 분포를 따르지 않음 (검정 통계량 1383.026 >= 임계값 0.576)
10.0% 수준에서 데이터는 정규 분포를 따르지 않음 (검정 통계량 1383.026 >= 임계값 0.656)
5.0% 수준에서 데이터는 정규 분포를 따르지 않음 (검정 통계량 1383.026 >= 임계값 0.787)
2.5% 수준에서 데이터는 정규 분포를 따르지 않음 (검정 통계량 1383.026 >= 임계값 0.918)
1.0% 수준에서 데이터는 정규 분포를 따르지 않음 (검정 통계량 1383.026 >= 임계값 1.092)


#### 2) 범주형 3집단 Kruskal Test

##### 분석 내용 : 회원의 연령대(범주형)에 따라 구매 빈도(연속형)가 차이가 있는가?
- 귀무 가설 : 빈도에 차이가 없다.
- 대립 가설 : 빈도에 차이가 있다.

In [110]:
# 연령대의 범주 2개 이상(3집단 이상)
age_list = df_saleshistory['연령대'].value_counts().index.to_list()
age_list

['40대', '50대', '30대이하', '60대', '70대이상']

In [115]:
series_list = list()
for age in age_list:
  condition = f'연령대 == "{age}"'
  series_frequency = saleshitory_pivot.query(condition)['구매빈도'] # condition에 따른 amount series 작성
  series_list.append(series_frequency)
series_list

[회원번호
 18942336      68
 21134272      24
 23485824      11
 26676288      21
 28871936      20
               ..
 1604454464    11
 1604731008    17
 1605182016     2
 1605532800     2
 1607149376     6
 Name: 구매빈도, Length: 4379, dtype: int64,
 회원번호
 19627200      361
 19727424       20
 19996544      315
 20258240        3
 20588608      388
              ... 
 1603270336     12
 1603821568     21
 1604866496      4
 1605495680     28
 1607346112      1
 Name: 구매빈도, Length: 2472, dtype: int64,
 회원번호
 76160960         2
 120020096        7
 139359616       24
 164094528       44
 185479360       24
               ... 
 1607360960       1
 1607535424       1
 1607672768       3
 1670400000    3586
 1670606016      12
 Name: 구매빈도, Length: 2055, dtype: int64,
 회원번호
 19391488        1
 20235968       23
 20636864      189
 22112384        5
 29868608        8
              ... 
 1599779200      9
 1599829312      2
 1600642240      2
 1600952192      3
 1601037568      9
 Name: 구매빈도, Leng

In [116]:
stats.kruskal(*series_list)

KruskalResult(statistic=36.260099823115866, pvalue=2.5582057450547897e-07)

In [117]:
round(2.5582057450547897e-07, 6)

0.0

In [120]:
# pvalue가 0.0 = 대립가설 채택 = 빈도에 차이가 있다. = 상관관계가 있다.

## 분석 결과 : 회원의 연령대(범주형)에 따라 구매 빈도(연속형)가 차이가 있다.
- 통계 결론 : 대립 가설 성립
- 사실 결론 : 연령대별 구매 빈도가 적은 연령대에게 특화된 프로모션 검토 필요.